In [2]:
#sample_leaf_options = [5,10,50,100,200,500]
#for leaf_size in sample_leaf_options :
#    model = RandomForestClassifier(n_estimators= 20, oob_score = True, n_jobs = -1,random_state =42, min_samples_leaf = leaf_size)
#    model.fit(df, y.values.ravel())
#    print(f'Out-of-bag score estimate: {model.oob_score_:.3}')

#redux features=18124 trees=200, min_samples_leaf=50 OOB 0.658
#redux features=18124 trees=200, min_samples_leaf=1 OOB 0.661 20min 41s
#redux features=18124 trees=200, min_samples_split=10 OOB 0.662 20min 41s
#redux features=18124 trees=200, min_samples_split=50 OOB 0.666 18min 55s
#redux features=18124 trees=200, min_samples_split=200 OOB 0.655 21min 36s
#base features=18124 trees=200, min_samples_leaf=50 OOB 0.658 total: 11min 58s
#base features=18124 trees=2000, min_samples_leaf=50 OOB 0.66 total: 1h 57min 19s
#redux+modelreduction(0.8*mean) features=7185 trees=200, min_samples_leaf=50 OOB 0.665
#redux+modelreduction(1*mean) features=5589 trees=200, min_samples_leaf=50 OOB 0.665
#redux+modelreduction(1.2*mean) features=4345 trees=200, min_samples_leaf=50 OOB 0.669
#redux+modelreduction(1.4*mean) features=3413 trees=200, min_samples_leaf=50 OOB 0.669
#redux+modelreduction(1.6*mean) features=2614 trees=200, min_samples_leaf=50 OOB 0.67
#redux+modelreduction(1.8*mean) features=2082 trees=200, min_samples_leaf=50 OOB 0.67
#redux+modelreduction(2*mean) features=1685 trees=200, min_samples_leaf=50 OOB 0.672
#redux+modelreduction(2.4*mean) features=1167 trees=200, min_samples_leaf=50 OOB 0.673
#redux+modelreduction(2.4*mean) features=1167 trees=2000, min_samples_leaf=50 OOB 0.676 7min 30s
#base+modelreduction(2.4*mean) features=940 trees=200, min_samples_leaf=50 OOB 0.672 37.1 s
#base+modelreduction(2.4*mean) features=940 trees=200, min_samples_leaf=1 OOB 0.676 30.6 s
#base+modelreduction(2.4*mean) features=940 trees=2000, min_samples_leaf=1 OOB 0.691 8min 48s
#base+modelreduction(2.4*mean) features=940 trees=2000, min_samples_leaf=15 OOB 0.686 total: 6min 58s

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.model_selection import train_test_split

def loadData():
    #df = pd.read_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/train_redux.csv')
    #y = pd.read_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/y.csv', sep=';', usecols=[1])
    df = pd.read_csv('E:\\Downloads\\train_concat_95_00001.csv')
    y = pd.read_csv('E:\\Downloads\\y.csv', sep=';', usecols=[1])
    print("Datas loaded")
    return df, y

df, y = loadData()
X_train, X_test, y_train, y_test = train_test_split(df, y.values.ravel(), test_size=.3)
print(df.shape)

Datas loaded
(26500, 6279)


In [6]:
def buildforest(train, y):
    forest = RandomForestClassifier(n_estimators=2000, oob_score=True, n_jobs=-1, min_samples_split=2, min_samples_leaf=5, random_state=42)
    forest.fit(train, y)
    print(f'Out-of-bag score estimate: {forest.oob_score_:.3}')
    return forest

%time forest = buildforest(X_train, y_train)
prob_pos = forest.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, prob_pos)
roc_auc = auc(fpr, tpr)
print('%s score: %0.2f ROC AUC: %0.2f' % (name, score, roc_auc))

Out-of-bag score estimate: 0.659
Wall time: 16min 3s


NameError: name 'roc_curve' is not defined

In [5]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(forest, threshold="2.4*mean", prefit=True)
dfmin = model.transform(df)
print(dfmin.shape)
print(df.shape)

(26500, 601)
(26500, 18124)


In [9]:
%time newforest = buildforest(dfmin, y)

Out-of-bag score estimate: 0.683
CPU times: user 29.8 s, sys: 1.69 s, total: 31.5 s
Wall time: 27.7 s


In [6]:
importances = forest.feature_importances_
coltodrop = []
#std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::]

#np.savetxt("/sgoinfre/goinfre/Perso/pdespres/ensdata/features_importances_2000.csv", indices, delimiter=",")
#np.savetxt('E:\\Downloads\\features_importances_2000.csv', indices, delimiter=",")
for f in range(18124):
    coltodrop.append([indices[f],importances[indices[f]]])
    #print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]*1000))
#df.drop(df.columns[coltodrop], axis=1, inplace=True)
#df.shape

In [24]:
dfmin.to_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/train_importance100.csv', index=False)

In [18]:
np.savetxt('E:\\Downloads\\features_importances_2000.csv', importances, delimiter=",")

In [7]:
test = pd.DataFrame(coltodrop)
test.to_csv('E:\\Downloads\\features_importances_2000.csv', index=False)

In [11]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

prob_pos = forest.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, prob_pos)
roc_auc = auc(fpr, tpr)
print('ROC AUC: %0.6f' % (roc_auc))

ROC AUC: 0.772443


In [12]:
test = pd.read_csv('E:\\Downloads\\test.csv')